<a href="https://colab.research.google.com/github/DAEN690-S22-OA/OceanicAirspaceProject/blob/ErickWorking/VerticalSeparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark
!pip install spark
!pip install geopy
!pip install pandasql

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 45 kB/s 
     |████████████████████████████████| 199 kB 55.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=ecf33a763a04640bb82c72ad99e721201ce57c00c3dfa71110f8e323355c7a82
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 41 kB 61 kB/s 
  Created wheel for spark: filename=spark-0.2.1-py3-none-any.whl size=58762 sha256=e0568931c078f275c18e625fdf2957a51c54f6c764b7c68a1831c864509faadd
  Stored in directory: /root/.cache/pip/wheels/4e/0e/f1/164619f9920fb447d294afaae11a7715bd442ded7225953d72
Successfully built spark
Looking in indexes: https://pypi.org/simple, https://us-p

In [ ]:
import pandas as pd
import pandasql as ps
from pyspark.sql import SparkSession
import os
import sys
import re
from datetime import datetime
from geopy.distance import geodesic as GD

In [ ]:
#Upload the full data file
from google.colab import files
uploadedfile = files.upload()


Saving airspaceData.csv to airspaceData.csv


In [ ]:
import io
airspaceDF = pd.read_csv(io.BytesIO(uploadedfile['airspaceData.csv']))

In [ ]:
print(airspaceDF)

                        DateTime   Hour   Minute   Second   microSecond  \
0        2021-12-24 23:59:59.992     23       59       59        992000   
1        2021-12-24 20:54:11.141     20       54       11        141000   
2        2021-12-24 12:36:02.773     12       36        2        773000   
3        2021-12-24 04:13:17.836      4       13       17        836000   
4        2021-12-24 23:59:59.984     23       59       59        984000   
...                          ...    ...      ...      ...           ...   
3303983  2021-12-24 12:36:02.859     12       36        2        859000   
3303984  2021-12-24 12:36:02.852     12       36        2        852000   
3303985  2021-12-24 12:36:02.805     12       36        2        805000   
3303986  2021-12-24 12:36:02.805     12       36        2        805000   
3303987  2021-12-24 12:36:02.789     12       36        2        789000   

          Latitude   Longitude   Height   TargetID    SelectedHeading  \
0        21.325208 -158.05

In [ ]:
#Some column names may be imported with leading spaces. This will remove that
airspaceDF.columns = [x.strip() for x in airspaceDF.columns]

print(airspaceDF.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3303988 entries, 0 to 3303987
Data columns (total 12 columns):
 #   Column           Dtype  
---  ------           -----  
 0   DateTime         object 
 1   Hour             int64  
 2   Minute           int64  
 3   Second           int64  
 4   microSecond      int64  
 5   Latitude         float64
 6   Longitude        float64
 7   Height           float64
 8   TargetID         object 
 9   SelectedHeading  float64
 10  TargetAddress    object 
 11  TrackNumber      float64
dtypes: float64(5), int64(4), object(3)
memory usage: 302.5+ MB
None


In [ ]:
#create SQL query for flights between the start and end time
sql1 = "SELECT * FROM airspaceDF WHERE Hour = 23 and Minute = 59 ORDER BY TargetID, Second"

#Run query and store results
records2359 = ps.sqldf(sql1, locals())

In [ ]:
#Export the file with just the 23:59 data
from google.colab import files

records2359.to_csv('records2359_1224.csv', encoding = 'utf-8-sig') 
files.download('records2359_1224.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**START HERE WITH records2359_1224.csv FILE**

In [ ]:
#Upload the file with the 23:59 data and save it as a dataframe named records2359

#Upload the records2359_1224.csv data file
from google.colab import files
uploadedfile = files.upload()

import io
records2359 = pd.read_csv(io.BytesIO(uploadedfile['records2359_1224.csv']))

In [ ]:
#Remove flights without a flight name or altitude
records2359 = records2359.dropna(subset=['TargetID'])
records2359 = records2359.dropna(subset=['Height'])

records2359.head(5)

,DateTime,Hour,Minute,Second,microSecond,Latitude,Longitude,Height,TargetID,SelectedHeading,TargetAddress,TrackNumber
16,2021-12-24 23:59:01.875,23,59,1,875000,21.085836,-157.526245,15400.0,AAH24,107.578125,A32F17,1234.0
17,2021-12-24 23:59:06.750,23,59,6,750000,21.081926,-157.517598,15400.0,AAH24,107.578125,A32F17,1234.0
18,2021-12-24 23:59:10.875,23,59,10,875000,21.078644,-157.510337,15400.0,AAH24,107.578125,A32F17,1234.0
19,2021-12-24 23:59:15.102,23,59,15,102000,21.075211,-157.502747,15425.0,AAH24,107.578125,A32F17,1234.0
20,2021-12-24 23:59:19.695,23,59,19,695000,21.071594,-157.494807,15400.0,AAH24,107.578125,A32F17,1234.0


In [ ]:
#Get the first entry for this minute of time
sql2 = "SELECT *, min(Second) FROM records2359 GROUP BY TargetID"

#Run query and store results
firstEntryDF = ps.sqldf(sql2, locals())
del firstEntryDF['min(Second)']

firstEntryDF.head(5)

,DateTime,Hour,Minute,Second,microSecond,Latitude,Longitude,Height,TargetID,SelectedHeading,TargetAddress,TrackNumber
0,2021-12-24 23:59:01.875,23,59,1,875000,21.085836,-157.526245,15400.0,AAH24,107.578125,A32F17,1234.0
1,2021-12-24 23:59:01.508,23,59,1,508000,19.976543,-155.128401,2625.0,AAH504,130.078125,A323F2,1272.0
2,2021-12-24 23:59:03.109,23,59,3,109000,25.290344,-147.790578,38775.0,AAL119,232.031250,AB6AFA,1024.0
3,2021-12-24 23:59:02.273,23,59,2,273000,42.300705,-132.887941,38200.0,AAL128,99.843750,AB7613,1024.0
4,2021-12-24 23:59:01.438,23,59,1,438000,33.262985,-123.632941,30500.0,AAL143,0.000000,A4FBAC,1024.0


In [ ]:
#Create tables to store proximity
sql3 = "SELECT * FROM firstEntryDF ORDER BY Latitude"
sql4 = "SELECT * FROM firstEntryDF ORDER BY Longitude"

#Run query and store results
LatitudeOrderDF = ps.sqldf(sql3, locals())
LongitudeOrderDF = ps.sqldf(sql4, locals())


In [ ]:
LatitudeOrderDF.head()

,DateTime,Hour,Minute,Second,microSecond,Latitude,Longitude,Height,TargetID,SelectedHeading,TargetAddress,TrackNumber
0,2021-12-24 23:59:04.047,23,59,4,47000,0.190538,-175.769980,37350.0,FDX77,210.937500,ABE1EB,1024.0
1,2021-12-24 23:59:00.625,23,59,0,625000,6.845460,-163.240356,36000.0,MGE600,NaN,AC3C99,1484.0
2,2021-12-24 23:59:03.977,23,59,3,977000,11.171885,178.954757,37225.0,ATN440,51.328125,AA21BF,1024.0
3,2021-12-24 23:59:01.453,23,59,1,453000,11.610489,-166.661035,42300.0,HAL451,NaN,A45233,1024.0
4,2021-12-24 23:59:04.836,23,59,4,836000,12.478409,142.616035,40650.0,SIA8890,258.750000,76B44A,1997.0


In [ ]:
LongitudeOrderDF.head()

,DateTime,Hour,Minute,Second,microSecond,Latitude,Longitude,Height,TargetID,SelectedHeading,TargetAddress,TrackNumber
0,2021-12-24 23:59:03.648,23,59,3,648000,17.969330,-176.990517,38025.0,MGE300,NaN,A9228A,1088.0
1,2021-12-24 23:59:04.047,23,59,4,47000,0.190538,-175.769980,37350.0,FDX77,210.937500,ABE1EB,1024.0
2,2021-12-24 23:59:02.883,23,59,2,883000,33.536041,-175.560315,40950.0,UAL903,288.281250,AA9321,1488.0
3,2021-12-24 23:59:00.430,23,59,0,430000,49.627487,-174.973249,34575.0,UAL893,265.078125,A31A73,1024.0
4,2021-12-24 23:59:00.109,23,59,0,109000,47.560684,-172.337723,40800.0,AAL9726,277.031250,AB5257,1024.0


In [ ]:
#Calculate the distance of the points closest to each other by longitidue and latitude

latInterest1 = []
latInterest2 = []
latDistance = []

longInterest1 = []
longInterest2 = []
longDistance = []

counter1 = 0
counter2 = 1

while counter2 < len(LongitudeOrderDF):
  #Create the coordinates that Geopy will ingest
  point0 = (LongitudeOrderDF['Latitude'][counter1], LongitudeOrderDF['Longitude'][counter1])
  point1 = (LongitudeOrderDF['Latitude'][counter2], LongitudeOrderDF['Longitude'][counter2])

  #Calculate the distance between the two points
  distance = GD(point0,point1).km

  #Assuming the distance threshold is 5KM
  if distance < 5:
    longInterest1.append(counter1)
    longInterest2.append(counter2)
    longDistance.append(distance)

  #Increase the count
  counter1 = counter1 + 1
  counter2 = counter2 + 1

print(longInterest1)
print(longInterest2)
print(longDistance)

print('')
print('---')
print('')

counter1 = 0
counter2 = 1

while counter2 < len(LatitudeOrderDF):
  #Create the coordinates that Geopy will ingest
  point0 = (LatitudeOrderDF['Latitude'][counter1], LatitudeOrderDF['Longitude'][counter1])
  point1 = (LatitudeOrderDF['Latitude'][counter2], LatitudeOrderDF['Longitude'][counter2])

  #Calculate the distance between the two points
  distance = GD(point0,point1).km

  #Assuming the distance threshold is 5KM
  if distance < 5:
    latInterest1.append(counter1)
    latInterest2.append(counter2)
    latDistance.append(distance)

  #Increase the count
  counter1 = counter1 + 1
  counter2 = counter2 + 1

print(latInterest1)
print(latInterest2)
print(latDistance)

[21, 24, 25, 26, 30, 31, 32, 41, 48, 49, 50, 71, 80, 103, 127, 138, 195]
[22, 25, 26, 27, 31, 32, 33, 42, 49, 50, 51, 72, 81, 104, 128, 139, 196]
[3.255579188739793, 2.1313517422297488, 1.434996466892325, 1.7472378177006196, 0.9218110577313594, 2.3903466132801245, 4.947650100035868, 4.12630797967906, 2.546336772937486, 1.4134566919894227, 4.886139494685955, 1.496699412626491, 2.3755551188193227, 3.080114252296092, 2.8429929977324573, 3.188905569758343, 4.048863482138465]

---

[10, 27, 49, 50, 58, 80, 88, 90, 91, 92, 132, 143, 207]
[11, 28, 50, 51, 59, 81, 89, 91, 92, 93, 133, 144, 208]
[3.080114252296092, 2.3755551188193227, 4.438961860041357, 2.3758468218133437, 1.4134566919894227, 0.9218110577313594, 3.255579188739793, 2.1313517422297488, 1.434996466892325, 1.7472378177006196, 2.8429929977324573, 3.188905569758343, 4.048863482138465]


In [ ]:
#Create a combined list of flights that came within 5 km of eachother at this time stamp

Resultsdf = pd.DataFrame(columns=['DateTime1','Hour1','Minute1','Second1','microSecond1','Latitude1','Longitude1','Height1',
                                  'TargetID1', 'SelectedHeading1', 'TargetAddress1', "TrackNumber2", 'DateTime2','Hour2','Minute2',
                                  'Second2','microSecond2','Latitude2','Longitude2','Height2', 'TargetID2', 'SelectedHeading2', 
                                  'TargetAddress2', "TrackNumber2", "Distance"])

print(Resultsdf)

Empty DataFrame
Columns: [DateTime1, Hour1, Minute1, Second1, microSecond1, Latitude1, Longitude1, Height1, TargetID1, SelectedHeading1, TargetAddress1, TrackNumber2, DateTime2, Hour2, Minute2, Second2, microSecond2, Latitude2, Longitude2, Height2, TargetID2, SelectedHeading2, TargetAddress2, TrackNumber2, Distance]
Index: []

[0 rows x 25 columns]


In [ ]:
#Adding the values from analyzing Longitude 
listlocation = 0

for x in longInterest1:
  position1 = longInterest1[listlocation]
  position2 = longInterest2[listlocation]
  
  entryValues1 = []

  position = 0
  while position < len(LongitudeOrderDF.columns):
    recordData = LongitudeOrderDF.loc[position1][position]
    entryValues1.append(recordData)
    position = position + 1

  position = 0
  while position < len(LongitudeOrderDF.columns):
    recordData = LongitudeOrderDF.loc[position2][position]
    entryValues1.append(recordData)
    position = position + 1

  entryValues1.append(longDistance[listlocation])

  Resultsdf.loc[len(Resultsdf.index)] = entryValues1

  listlocation = listlocation + 1

In [ ]:
#Adding the values from analyzing Latitude 
listlocation = 0

for x in latInterest1:
  position1 = latInterest1[listlocation]
  position2 = latInterest2[listlocation]
  
  entryValues1 = []

  position = 0
  while position < len(LatitudeOrderDF.columns):
    recordData = LatitudeOrderDF.loc[position1][position]
    entryValues1.append(recordData)
    position = position + 1

  position = 0
  while position < len(LatitudeOrderDF.columns):
    recordData = LatitudeOrderDF.loc[position2][position]
    entryValues1.append(recordData)
    position = position + 1

  entryValues1.append(latDistance[listlocation])

  Resultsdf.loc[len(Resultsdf.index)] = entryValues1

  listlocation = listlocation + 1

In [ ]:
print(Resultsdf)

                  DateTime1 Hour1 Minute1 Second1 microSecond1  Latitude1  \
0   2021-12-24 23:59:01.719    23      59       1       719000  22.160156   
1   2021-12-24 23:59:14.414    23      59      14       414000  22.208908   
2   2021-12-24 23:59:02.523    23      59       2       523000  22.216095   
3   2021-12-24 23:59:02.898    23      59       2       898000  22.228880   
4   2021-12-24 23:59:01.234    23      59       1       234000  21.955398   
5   2021-12-24 23:59:13.711    23      59      13       711000  21.963135   
6   2021-12-24 23:59:08.648    23      59       8       648000  21.984558   
7   2021-12-24 23:59:02.023    23      59       2        23000  21.321670   
8   2021-12-24 23:59:02.586    23      59       2       586000  21.306839   
9   2021-12-24 23:59:23.148    23      59      23       148000  21.322042   
10  2021-12-24 23:59:02.078    23      59       2        78000  21.323066   
11  2021-12-24 23:59:08.688    23      59       8       688000  21.155777   

In [ ]:
heightDifference = []
potentialLoss = []

counter = 0

while counter < len(Resultsdf):
  difference = abs((Resultsdf['Height1'][counter]) - (Resultsdf['Height2'][counter]))
  heightDifference.append(difference)

  if difference <= 1000:
    potentialLoss.append('True')
  else:
    potentialLoss.append('False')

  counter = counter + 1

print(heightDifference)
print(potentialLoss)

[650.0, 200.0, 750.0, 200.0, 800.0, 1400.0, 475.0, 3050.0, 0.0, 275.0, 75.0, 175.0, 1025.0, 575.0, 7950.0, 1925.0, 5000.0, 575.0, 1025.0, 4150.0, 125.0, 275.0, 800.0, 650.0, 200.0, 750.0, 200.0, 7950.0, 1925.0, 5000.0]
['True', 'True', 'True', 'True', 'True', 'False', 'True', 'False', 'True', 'True', 'True', 'True', 'False', 'True', 'False', 'False', 'False', 'True', 'False', 'False', 'True', 'True', 'True', 'True', 'True', 'True', 'True', 'False', 'False', 'False']


In [ ]:
Resultsdf['HeightDifference'] = heightDifference
Resultsdf['PotentialLoss'] = potentialLoss

print(Resultsdf)

                  DateTime1 Hour1 Minute1 Second1 microSecond1  Latitude1  \
0   2021-12-24 23:59:01.719    23      59       1       719000  22.160156   
1   2021-12-24 23:59:14.414    23      59      14       414000  22.208908   
2   2021-12-24 23:59:02.523    23      59       2       523000  22.216095   
3   2021-12-24 23:59:02.898    23      59       2       898000  22.228880   
4   2021-12-24 23:59:01.234    23      59       1       234000  21.955398   
5   2021-12-24 23:59:13.711    23      59      13       711000  21.963135   
6   2021-12-24 23:59:08.648    23      59       8       648000  21.984558   
7   2021-12-24 23:59:02.023    23      59       2        23000  21.321670   
8   2021-12-24 23:59:02.586    23      59       2       586000  21.306839   
9   2021-12-24 23:59:23.148    23      59      23       148000  21.322042   
10  2021-12-24 23:59:02.078    23      59       2        78000  21.323066   
11  2021-12-24 23:59:08.688    23      59       8       688000  21.155777   

In [ ]:
Resultsdf.to_csv('results2359.csv', encoding = 'utf-8-sig') 
files.download('results2359.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#Get the first entry for this minute of time
sql6 = "SELECT * FROM Resultsdf WHERE PotentialLoss = 'True' "

#Run query and store results
LossCandidates = ps.sqldf(sql6, locals())

In [ ]:
print(LossCandidates)

                  DateTime1  Hour1  Minute1  Second1  microSecond1  Latitude1  \
0   2021-12-24 23:59:01.719     23       59        1        719000  22.160156   
1   2021-12-24 23:59:14.414     23       59       14        414000  22.208908   
2   2021-12-24 23:59:02.523     23       59        2        523000  22.216095   
3   2021-12-24 23:59:02.898     23       59        2        898000  22.228880   
4   2021-12-24 23:59:01.234     23       59        1        234000  21.955398   
5   2021-12-24 23:59:08.648     23       59        8        648000  21.984558   
6   2021-12-24 23:59:02.586     23       59        2        586000  21.306839   
7   2021-12-24 23:59:23.148     23       59       23        148000  21.322042   
8   2021-12-24 23:59:02.078     23       59        2         78000  21.323066   
9   2021-12-24 23:59:08.688     23       59        8        688000  21.155777   
10  2021-12-24 23:59:01.773     23       59        1        773000  19.419499   
11  2021-12-24 23:59:04.633 